In [1]:
import os
import csv
import glob
import json
import pickle
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from typing import List, Tuple, Any

import multiprocessing
from multiprocessing import Pool

import dask
from dask.distributed import wait
from dask.distributed import Client, wait, LocalCluster

In [2]:
# set n_workers to number of cores
client = Client(n_workers=multiprocessing.cpu_count(), threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:34677 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 7.23 GiB


In [3]:
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}


minCount = 1
rssiFillerValue = -999.0
dtFillerValue   = 1000.0
freqFillerValue = 0
outputDir = '.'
sampleCsvPath = 'sample_submission.csv'
buildingBssidPklFilePath = "buildingBssids.pkl"

In [4]:
def input_dir() -> Path:
    return Path('.')

def generate_target_buildings() -> List[str]:
    ssubm = pd.read_csv(sampleCsvPath)
    ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
    return sorted(ssubm_df[0].value_counts().index.tolist()) # type: ignore

def extract_wps_wifis(file: Path) -> Tuple[List[str], List[str]]:
    wps = []
    wifis = []
    with open(file) as f:
        for row in csv.reader(f, delimiter="\t", doublequote=True):
            if row[1] == "TYPE_WAYPOINT":
                # x
                row[2] = float(row[2])  # type: ignore
                # y
                row[3] = float(row[3])  # type: ignore
                wps.append([int(row[0]), row[2], row[3]])
            elif row[1] == "TYPE_WIFI":
                # wifi signal value
                row[4] = int(row[4])  # type: ignore
                wifis.append(row)
    wps = sorted(wps, key=lambda x: x[0])  # timestamp
    wifis = sorted(wifis, key=lambda x: x[0])  # timestamp
    return wps, wifis

In [5]:
def generateFloorBssids_train(building : str):
    """
    for a given building in train set, this function calculates
    the unique wifi bssids within each floor
    
    returns :  a dict with keys = {building}_{floor}, values = list(set(floorBssids))
    """
    building_path = input_dir() / 'train' / building
    floorBssids = {}
    folders = sorted(building_path.glob('*'))
    for folder in folders:
        folderData = []
        files = folder.glob("*.txt")
        for file in files:
            _, wifiData = extract_wps_wifis(file)
            folderData.extend([t[3] for t in wifiData])
        floorBssids[f"{folder.name}"] = list(set(folderData))
    return floorBssids

In [6]:
def generateIntersectingBssids(floorBssids):
    """
    given a dict with floor -> bssid list mapping,
    returns : list of bssids which occur in more than 1 floor 
    """
    commonBssids = []
    for k1, v1 in floorBssids.items():
        for k2, v2 in floorBssids.items():
            if (k1 != k2):
                intersectingBssids = list(set(v1).intersection(set(v2)))
                commonBssids.extend(intersectingBssids)
    return commonBssids

In [7]:
def printBssidsInfo(floorBssids):
    """
    given a dict with floor -> bssid list mapping,m
    Function calcuates total number of bssids and number 
    of unique bssids and prints result
    """
    totalBssids = []
    for k,v in floorBssids.items():
        ## print(f"{k} has {len(v)} total bssids")
        totalBssids.extend(v)
    
    print(f"Totally, There are {len(totalBssids)} bssids")
    print(f"There are {len(set(totalBssids))} unique bssids")

In [8]:
def generateUniqueFloorBssids(floorBssids, commonBssids):
    """
    floorBssids : a dict with floor -> bssid list mapping
    commonBssids : list of bssids which are present in more than one floor
    """
    uniqueFloorBssids = {}
    for k,v in floorBssids.items():
        uniqueFloorBssids[k] = list(set(v) - set(commonBssids))    
    return uniqueFloorBssids

In [9]:
def generateFloorUniqueBssidData(buildingsList):
    floorUniqueBssidData = {}
    for building in buildingsList:
        floorBssids = generateFloorBssids_train(building)    # consumes most time
        commonBssids = generateIntersectingBssids(floorBssids)
        uniqueBssids = generateUniqueFloorBssids(floorBssids, commonBssids)

        print(building)
        print(f"building unique bssids information")
        printBssidsInfo(floorBssids)
        print(f"Floor unique bssids information")
        printBssidsInfo(uniqueBssids)
        print('-----------------------------------')

        floorUniqueBssidData[building] = uniqueBssids
    
    with open("floorUniqueBssidData.json", "w") as outfile: 
        json.dump(floorUniqueBssidData, outfile)

In [10]:
class wifiAPFloorMapping:
    def __init__(self, building, floorUniqueBssidMapPath):
        self.building = building
        self.floorList = self.getBuildingFloorList()
        self.uniqueBssidsMap = self.getFloorUniqueBssids(floorUniqueBssidMapPath)
        self.outputData = self.generateOutputData()
    
    def getFloorUniqueBssids(self, floorUniqueBssidMapPath):
        with open(floorUniqueBssidMapPath, "r") as infile:
            floorUniqueBssidMap = json.load(infile)
        return floorUniqueBssidMap.get(self.building, None)
    
    def getBuildingFloorList(self):
        buildingPath = input_dir() / 'train' / self.building
        folders = sorted(buildingPath.glob('*'))
        return [folder.name for folder in folders] + ['common']

    def generateOutputData(self):
        outputData = {'pathName' : []}
        for floor in self.floorList:
            outputData[f"{floor}_count"] = []
            outputData[f"{floor}_mean"]  = []
            outputData[f"{floor}_median"]  = []
        return outputData
    
    def findMappingFloor(self, wifiAp):
        matchingfloor = 'common'
        if self.uniqueBssidsMap is not None:
            for floor, floorAPList in self.uniqueBssidsMap.items():
                if wifiAp in floorAPList:
                    matchingfloor = floor
                    break
        return matchingfloor 
    
    def getPathFileWiFiData(self, file):
        _, wifiData = extract_wps_wifis(file)
        wifiData = pd.DataFrame(wifiData, columns = ['timestamp','type','ssid', 'bssid', 'rssi', 'freq', 'last_ts'])
        wifiData.drop(labels=['timestamp', 'type', 'ssid', 'freq', 'last_ts'], axis=1, inplace=True)
        wifiData['mappedFloor'] = wifiData['bssid'].apply(self.findMappingFloor)
        return wifiData
        
    def updatePathFileToOutput(self, file):
        # add path file to output
        self.outputData['pathName'].append(file.name.split('.')[0])
        
        # create wifi data of pathfile
        wifiData = self.getPathFileWiFiData(file)
        
        # shortlist wifi bssids based on each floor list
        for floor in self.floorList:
            rssiData = wifiData[wifiData['mappedFloor']== floor]['rssi'].values
            if len(rssiData) > 0:
                mean, median = np.mean(rssiData), np.median(rssiData)
            else:
                mean, median = 0.0, 0.0
                
            self.outputData[f"{floor}_count"].append(len(rssiData))
            self.outputData[f"{floor}_mean"].append(mean)
            self.outputData[f"{floor}_median"].append(median)
        del wifiData

In [11]:
ssubm = pd.read_csv(sampleCsvPath)
ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
buildingsList = generate_target_buildings()

In [14]:
building = buildingsList[0]
building_path = input_dir() / 'test'
print(building_path)
buildingTestPathFiles = ssubm_df[ssubm_df[0] == building][1].unique()
print(f"There are {len(buildingTestPathFiles)} test path files in building")

test
There are 29 test path files in building


In [15]:
temp = wifiAPFloorMapping(building, 'floorUniqueBssidData.json')

In [20]:
for testPathFile in buildingTestPathFiles:
    temp.updatePathFileToOutput(building_path / f"{testPathFile}.txt")

In [21]:
outputDf = pd.DataFrame(temp.outputData)

In [22]:
outputDf

,pathName,B1_count,B1_mean,B1_median,F1_count,F1_mean,F1_median,F2_count,F2_mean,F2_median,F3_count,F3_mean,F3_median,F4_count,F4_mean,F4_median,common_count,common_mean,common_median
0,046cfa46be49fc10834815c6,8,-81.000000,-81.0,373,-80.184987,-79.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,13531,-72.579632,-73.0
1,05d052dde78384b0c543d89c,0,0.000000,0.0,289,-81.519031,-82.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,10366,-78.294906,-80.0
2,0c06cc9f21d172618d74c6c8,6218,-77.754101,-80.0,0,0.000000,0.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,4158,-72.786436,-76.0
3,146035943a1482883ed98570,4254,-74.612365,-78.0,0,0.000000,0.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,513,-79.105263,-81.0
4,1ef2771dfea25d508142ba06,0,0.000000,0.0,297,-80.259259,-81.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,13101,-77.358217,-78.0
5,3506b3b626f494b0f0b934ca,36,-71.666667,-71.0,566,-74.696113,-75.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,17662,-73.659552,-74.0
6,3e0aebb66ef39150bbc27c24,0,0.000000,0.0,768,-80.802083,-84.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,12347,-76.187090,-78.0
7,3e1d46017fbfcc8136bd1e9b,0,0.000000,0.0,240,-74.808333,-73.5,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,16981,-73.792238,-76.0
8,412da1891c4780f6f0f7f4bc,0,0.000000,0.0,573,-75.141361,-77.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,14153,-76.769307,-78.0
9,6d89334316127640cff99800,7595,-76.129559,-77.0,0,0.000000,0.0,0,0.000000,0.0,0,0.0000,0.0,0,0.0,0.0,3386,-79.509451,-85.0


In [23]:
outputDf.to_csv('testOuptutFloorPredictions.csv')